In [27]:
from coffea.nanoevents import NanoEventsFactory, FCC, BaseSchema

events = NanoEventsFactory.from_root(
    "../../../../data/eos/experiment/fcc/ee/generation/DelphesEvents/winter2023/IDEA/p8_ee_ZZ_ecm240/events_136205668.root:events",
    schemaclass=FCC.get_schema("pre-edm4hep1"),
    # schemaclass=BaseSchema,
    # entry_stop=100,
    delayed=True,
    uproot_options = {"filter_name": lambda x : "PARAMETERS" not in x}
).events()

In [28]:
# Helper functions
import awkward as ak
import numpy as np
import vector
vector.register_awkward()

In [29]:
# 1. To replace: FCCAnalyses::ZHfunctions::resonanceBuilder_mass(91.2,false)(selected_muons, MCRecoAssociations0, MCRecoAssociations1, ReconstructedParticles, Particle, Particle0, Particle1)
def resonanceBuilder_mass(resonance_mass=None, use_MC_Kinematics=False, leptons=None, MCRecoAssociations=None, ReconstructedParticles=None, MCParticles=None):
    '''
    Build the Z resonance based on the available leptons. Returns the best lepton pair compatible with the Z mass and recoil at 125 GeV
    technically, it returns a ReconstructedParticleData object with index 0 the di-lepton system, index and 2 the leptons of the pair
    '''
    if leptons is None:
        raise AttributeError("No leptons passed")
    #Create all the combinations
    combs = ak.combinations(leptons,2)
    # Get dileptons
    lep1 , lep2 = ak.unzip(combs)
    di_lep = lep1 + lep2 # This process drops any other field except 4 momentum fields

    di_lep["charge"] =  lep1.charge + lep2.charge
    di_lep["l1_index"] = lep1.index
    di_lep["l2_index"] = lep2.index

    # Choose oppositely charged leptons
    di_lep = di_lep[di_lep.charge == 0]

    # Sort by closest mass to the resonance value
    sort_mask = ak.argsort(abs(resonance_mass-di_lep.mass), axis=1)
    Reso = di_lep[sort_mask]

    return Reso

In [30]:
# 2. To replace: FCCAnalyses::ZHfunctions::getTwoHighestPMuons(rest_of_muons)") # Find the higest p muon pair from the remaining muons (off-shell Z)
def getTwoHighestPMuons(muons):
    '''
    Sort by decending P and return the pair of particles with highest P and opposite charges
    '''
    # if not ak.all(ak.num(muons, axis=1) > 1 ):
    #     raise IndexError("Need at least two particles!")
    sorted_muons_p = ak.argsort(muons.p, ascending=False)
    sorted_muons = muons[sorted_muons_p]

    # First particle is always selected, if the second one has the opposite charge, then its accepted otherwise we move on to the third and so on
    # Interestingly, this type of operation is non trivial in an array format
    first_muon, other_muons = sorted_muons[:, 0:1], sorted_muons[:, 1:]

    # prepare before cartesian : replace none with []
    first_muon = ak.fill_none(first_muon, [], axis=0)
    other_muons = ak.fill_none(other_muons, [], axis=0)
    # All combinations
    all_comb = ak.cartesian([first_muon, other_muons])
    l1, l2 = ak.unzip(all_comb)
    charge_mask = l1.charge!= l2.charge
    l1 = l1[charge_mask]
    l2 = l2[charge_mask]

    at_least_one_opp_charged = ak.sum(charge_mask, axis=1) > 0
    
    return ak.firsts(l1), ak.firsts(l2), at_least_one_opp_charged

In [31]:
# Main calculations
Muons = events.ReconstructedParticles[events.Muonidx0.index]
Muons["index"] = events.Muonidx0.index # Attach the reco index for easier calculations later

In [32]:
sel_muon = Muons.p > 2.0
selected_muons_p = Muons[sel_muon]

In [33]:
at_least_4_muons = ak.num(selected_muons_p, axis=1) > 3
selected_muons = selected_muons_p[at_least_4_muons]

In [34]:
# Build Z resonances
Z = resonanceBuilder_mass(resonance_mass=91.2, use_MC_Kinematics=False, leptons=selected_muons)

In [35]:
# On Shell Z
zll = ak.firsts(Z)

In [64]:
l1 = ak.firsts(selected_muons[selected_muons.index == zll.l1_index])
l2 = ak.firsts(selected_muons[selected_muons.index == zll.l2_index])

In [36]:
def create_mask(a, b, c):
    mask1 = a != c
    mask2 = b != c
    mask = mask1 & mask2
    return mask

In [37]:
mask = create_mask(zll.l1_index, zll.l2_index, selected_muons.index)

In [38]:
rest_of_muons = selected_muons[mask]

In [39]:
m1, m2, c_mask = getTwoHighestPMuons(rest_of_muons)

In [40]:
non_res_Z = m1 + m2
# Angle between the two
non_res_Z_angle = m1.deltaangle(m2)

In [73]:
# Collect all the four Muons
fourMuons_collected = ak.concatenate(
    (
        ak.drop_none(l1[c_mask])[:, np.newaxis],
        ak.drop_none(l2[c_mask])[:, np.newaxis],
        ak.drop_none(m1)[:, np.newaxis],
        ak.drop_none(m2)[:, np.newaxis]
    ),
    axis=1
)

In [74]:
fourMuons_p = fourMuons_collected.p.compute()

In [75]:
fourMuons_p

<Array [[74.1, 62.4, 88.4, 15.5], ..., [...]] type='213 * 4 * float32[param...'>

In [76]:
fourMuons_collected = ak.mask(fourMuons_collected, ak.num(fourMuons_collected, axis=1) > 3)
fourMuons = ak.mask(zll, c_mask) + non_res_Z

In [77]:
non_res_Z_m = non_res_Z.m.compute()

In [78]:
zll_m = zll[c_mask].m.compute()

In [79]:
fourMuons_m = fourMuons.m.compute()

In [80]:
intLumi        = 10.80e+06 #in pb-1
scale = 52.6539*intLumi/100000
import hist

In [81]:
res_Z_hist = hist.Hist.new.Regular(50,0,250).Double().fill(ak.drop_none(zll_m))*scale
non_res_Z_hist = hist.Hist.new.Regular(50,0,250).Double().fill(ak.drop_none(non_res_Z_m))*scale
four_muon_mass_hist = hist.Hist.new.Regular(50,0,250).Double().fill(ak.drop_none(fourMuons_m))*scale

In [82]:
res_Z_hist

Hist(Regular(50, 0, 250, label='Axis 0'), storage=Double()) # Sum: 1211250.3155999999

In [83]:
non_res_Z_hist

Hist(Regular(50, 0, 250, label='Axis 0'), storage=Double()) # Sum: 1211250.3155999999

In [84]:
four_muon_mass_hist

Hist(Regular(50, 0, 250, label='Axis 0'), storage=Double()) # Sum: 1211250.3155999999